In [90]:
# System imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import panel as pn
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets
from string import digits
import csv
import json
import numpy as np
import dateparser
import panel as pn

pn.extension()

%matplotlib inline

In [91]:
# Local imports
import sys
sys.path.append("../lib2")

from Constants import Constants
from ProcessingTools import DateProcessingTools

In [92]:
# Construct the tools
debug_level = 0
constants = Constants()
tool_data_processing = DateProcessingTools(debug_level)


In [93]:
# Import all preprocessed data
atlas           = pd.read_pickle(constants.PREPROCESSED_ATLAS_FILE_PATH)
forecast        = pd.read_pickle(constants.PREPROCESSED_FORECAST_DATA_FILE_PATH)
revenue2020     = pd.read_pickle(constants.PREPROCESSED_REVENUE2020_FILE_PATH)
revenue2020A    = pd.read_pickle(constants.PREPROCESSED_REVENUE2020A_FILE_PATH)
atlas2          = pd.read_pickle(constants.PREPROCESSED_ATLAS_2_FILE_PATH)


In [94]:
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")

In [95]:
atlas2.head()

,Number of Users,Invoice Date,Invoice #,Invoice Amount,Subscription,Account Code,Address,Lat,Long,Service Start,Service End,Subscription Duration
Customers,,,,,,,,,,,,
University 1,NaN,2015-03-20 00:00:00-04:00,ATLAS 315,72000.0,1 Year,4700-0-00-00000-18-0000,"1201 N 3rd St #6, Baton Rouge, LA 70802",30.460190,-91.188530,2015-03-18 00:00:00-04:00,2016-06-30 00:00:00-04:00,365 days
University 2,250.0,2015-05-28 00:00:00-04:00,AJ501,3500.0,1 Year,4700-0-00-00000-16-0000,"1 Kellogg Cir, Emporia, KS 66801",38.392609,-96.181396,2015-06-01 00:00:00-04:00,2016-06-30 00:00:00-04:00,365 days
University 3,100.0,2015-06-23 00:00:00-04:00,AJ502,3500.0,1 Year,4700-0-00-00000-17-0000,"150 University Blvd, Morehead, KY 40351",38.184921,-83.434441,2015-06-10 00:00:00-04:00,2016-06-30 00:00:00-04:00,365 days
University 4,NaN,2015-06-26 00:00:00-04:00,AJ503,6500.0,1 Year,4700-0-00-00000-32-0000,"44 Pierrepont Ave, Potsdam, NY 13676",44.663780,-74.978409,2015-06-01 00:00:00-04:00,2016-06-30 00:00:00-04:00,365 days
University 5,NaN,2015-10-07 00:00:00-04:00,AJ504,750.0,1 Year,4700-0-00-00000-20-0000,"47645 College Dr, St Marys City, MD 20686",38.190601,-76.426300,2015-10-05 00:00:00-04:00,2016-09-30 00:00:00-04:00,365 days


In [87]:
# Adding New Columns to categorize data by Year, Month, and each Quarter
year_column = []
month_column = []
quarter_column = []
for date in ATLAS_with_fixed_dates["Start date"]:
    year_column.append(date.year)
    month_column.append(date.month)
    quarter_column.append(date.quarter)
    
# Add appended to the Data Frame (Start Date or Invoice Date)
periods_df = pd.DataFrame({"Year":year_column,"Monthly": month_column, "Quarterly": quarter_column})
periods_df.head()
ATLAS_with_periods = pd.concat([ATLAS.reset_index(), periods_df], axis='columns', join="inner")
ATLAS_with_periods.head()

######

year_column = []
month_column = []
quarter_column = []
for date in atlas2["Service Start"]:
    year_column.append(date.year)
    month_column.append(date.month)
    quarter_column.append(date.quarter)
    
# Add appended to the Data Frame (Start Date or Invoice Date)
periods_df = pd.DataFrame({"Year":year_column,"Monthly": month_column, "Quarterly": quarter_column})
atlas2.reset_index()
atlas2.head()

,Number of Users,Invoice Date,Invoice #,Invoice Amount,Subscription,Account Code,Address,Lat,Long,Service Start,Service End,Subscription Duration
Customers,,,,,,,,,,,,
University 1,NaN,2015-03-20 00:00:00-04:00,ATLAS 315,72000.0,1 Year,4700-0-00-00000-18-0000,"1201 N 3rd St #6, Baton Rouge, LA 70802",30.460190,-91.188530,2015-03-18 00:00:00-04:00,2016-06-30 00:00:00-04:00,365 days
University 2,250.0,2015-05-28 00:00:00-04:00,AJ501,3500.0,1 Year,4700-0-00-00000-16-0000,"1 Kellogg Cir, Emporia, KS 66801",38.392609,-96.181396,2015-06-01 00:00:00-04:00,2016-06-30 00:00:00-04:00,365 days
University 3,100.0,2015-06-23 00:00:00-04:00,AJ502,3500.0,1 Year,4700-0-00-00000-17-0000,"150 University Blvd, Morehead, KY 40351",38.184921,-83.434441,2015-06-10 00:00:00-04:00,2016-06-30 00:00:00-04:00,365 days
University 4,NaN,2015-06-26 00:00:00-04:00,AJ503,6500.0,1 Year,4700-0-00-00000-32-0000,"44 Pierrepont Ave, Potsdam, NY 13676",44.663780,-74.978409,2015-06-01 00:00:00-04:00,2016-06-30 00:00:00-04:00,365 days
University 5,NaN,2015-10-07 00:00:00-04:00,AJ504,750.0,1 Year,4700-0-00-00000-20-0000,"47645 College Dr, St Marys City, MD 20686",38.190601,-76.426300,2015-10-05 00:00:00-04:00,2016-09-30 00:00:00-04:00,365 days


In [88]:
# Calculate the mean values for each neighborhood

total_data = atlas2.groupby(["Customers"])["Service Start"]
total_data


In [89]:

total_data.hvplot(kind='line', figsize=(20, 10))


AttributeError: 'SeriesGroupBy' object has no attribute 'hvplot'

In [ ]:

# Construct a correlation table
Correlation = Combined_Returns.corr()
Correlation
sns.heatmap(Correlation, vmin=-1, vmax=1)

In [ ]:
def neighborhood_map():
    """Neighborhood Map"""
     = atlas2.groupby(["Customers"]).mean()
    mean_data.reset_index(inplace=True)
    px.set_mapbox_access_token(mapbox_token)
    neighborhood_map_plot = px.scatter_mapbox(
            mean_data,
            lat="Lat",
            lon="Long",
            size="Subscription",
            color="Customer"
        )
    return neighborhood_map_plot
neighborhood_map()